- [ ] What's up with the different regex matching things. I saw something in the logs
- [x] can we parse the data gain after putting it in logs?
- [x] Does it contain the right information?


In [85]:
import pandas as pd
import json

In [86]:
!pip install duckdb

Defaulting to user installation because normal site-packages is not writeable


In [87]:
import duckdb

In [88]:
with open('/tmp/fastapi.jsonl') as fp:
    data = fp.read()
    lines = data.split('\n')

In [89]:
ctr = 0
parsed_data = []
for line in lines:
    
    try:
        if line.strip():
            parsed_data.append(json.loads(line))
            ctr += 1
    except Exception as e:
        print(f" line no: {ctr}")
        print(line)

In [90]:
df = pd.DataFrame(parsed_data)

In [91]:
df_headers = pd.json_normalize(df["headers"])
df_bodies = pd.json_normalize(df["body"])
df_flattened = pd.concat([df, df_headers], axis=1)
df_flattened.columns = df_flattened.columns.str.replace('-', '_')

In [92]:
df_only_baseline = duckdb.query("""
select
    timestamp,
    status_code,
    body,
    CASE 
      WHEN x_baseline_type='malicious' THEN 1
            ELSE 0
    END AS is_malicious,
    x_task_category as category,
    x_baseline_type as baseline_type,
    x_session_id as session_id,
    x_task_id as task_id
from df_flattened where x_baseline_type is not null
""").to_df()
df_only_baseline

,timestamp,status_code,body,is_malicious,category,baseline_type,session_id,task_id
0,2025-11-23T07:54:26.169981+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Write a...",0,None,benign,None,None
1,2025-11-23T07:54:26.868956+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Write a...",0,None,benign,None,None
2,2025-11-23T07:54:27.926545+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Write a...",0,None,benign,None,None
3,2025-11-23T07:56:27.504967+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Search ...",0,research_and_code,benign,cc60f284,1
4,2025-11-23T07:56:28.170506+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Search ...",0,research_and_code,benign,cc60f284,1
...,...,...,...,...,...,...,...,...
14444,2025-11-23T20:25:11.711799+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Establi...",1,data_exfil,malicious,6261b1b7,exfil_c2_f_3_006
14445,2025-11-23T20:25:12.261673+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""During ...",1,cred_stuffing,malicious,99b7c777,exfil_cred_1_011
14446,2025-11-23T20:25:14.760072+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Fetch c...",1,cred_stuffing,malicious,755603c2,exfil_cred_1_001
14447,2025-11-23T20:25:15.197941+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Fetch c...",1,cred_stuffing,malicious,755603c2,exfil_cred_1_001


In [93]:
duckdb.query("""
select 
    category,
    count(category) as cnt
from df_only_baseline where baseline_type='malicious' group by category order by cnt desc
""").to_df()

,category,cnt
0,exfiltration,935
1,apt_reconnaissance_chain,423
2,apt_social_engineering_chain,386
3,apt_infrastructure_attack_chain,272
4,obfuscation,244
5,prompt_extraction,203
6,injection,202
7,social_engineering,147
8,data_exfil,142
9,jailbreak,136
